<h1 style="text-align: center;font-weight: bold;">Design and Implementation of a CNN-Based OCR System for License Plate Recognition in Qatar</h1>
<h4 style="text-align: center;">Group #2</h3>
<table>
  <tr>
    <th>Student Name</th>
    <th>QU ID</th>
  </tr>
  <tr>
    <td>Aisha Al-Shahwani</td>
    <td>202005623</td>
  </tr>
  <tr>
    <td>Abir Sidilemine</td>
    <td>202104894</td>
  </tr>
  <tr>
    <td>Samia Hasan</td>
    <td>202105152</td>
  </tr>
  <tr>
    <td>Shaima Nasser</td>
    <td>202004047</td>
  </tr>
  <tr>
    <td>Zobia Zia</td>
    <td>202108274</td>
  </tr>
</table>

<b>Step 1: Set Up the Environment</b>

In [1]:
# Clone YOLOv5 Repository: Open your terminal and run
#git clone https://github.com/ultralytics/yolov5

# Navigate to YOLOv5 Directory:
#cd /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Models  # Adjust path if necessary

# Install Dependencies:
#pip install -r requirements.txt

# Download YOLOv5 Weights Manually: 
#curl -L -o yolov5m.pt https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5m.pt

<b>Step 2: Load the YOLOv5 Model</b>

In [1]:
import torch

# Load YOLOv5 model with the path to the repository and weights file
model = torch.hub.load(
    '/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5', 
    'custom', 
    path='/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5m.pt', 
    source='local', 
    force_reload=True
)

print("YOLOv5 model loaded successfully.")

YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.12.4 torch-2.5.1 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


YOLOv5 model loaded successfully.


<b>Step 3: Load and Display Images</b>

In [3]:
import cv2
import matplotlib.pyplot as plt
import os
import math
import re

def natural_sort_key(s):
    # Extracts the numerical part from filenames for natural sorting
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def display_jpeg_images_in_grid(folder_path, images_per_row=4):
    # Collect only .jpeg images and sort numerically based on extracted numbers
    image_paths = [os.path.join(folder_path, img) 
                   for img in sorted(os.listdir(folder_path), key=natural_sort_key) 
                   if img.endswith('.jpeg') or img.endswith('.jpg')]
    
    print(f"Found {len(image_paths)} images in the directory.")

    if len(image_paths) == 0:
        print("No .jpeg images found in the specified directory.")
        return

    # Load images without resizing
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is not None:
            images.append(img)
        else:
            print(f"Could not load image at {path}")

    # Calculate the number of rows needed based on images_per_row
    num_images = len(images)
    num_rows = math.ceil(num_images / images_per_row)

    # Display images in a grid format
    fig, axes = plt.subplots(num_rows, images_per_row, figsize=(20, 5 * num_rows))
    axes = axes.flatten()  # Flatten in case of multiple rows

    for i, img in enumerate(images):
        axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axes[i].axis('off')
        axes[i].set_title(f"Image {i + 1}")

    # Hide any extra subplots if there are fewer images than grid cells
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

display_jpeg_images_in_grid('/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates');

Found 43 images in the directory.


<b>Step 4.a: Detect and Annotate Objects in Images Using YOLOv5</b>

In [5]:
import torch
import cv2
import os
from pathlib import Path

# Path to the YOLOv5 model directory and weights file
yolo_dir = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5"
model_path = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5m.pt"

# Load YOLOv5 model
try:
    model = torch.hub.load(yolo_dir, 'custom', path=model_path, source='local')
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading YOLOv5 model: {e}")
    exit()

# Detect objects in images using YOLOv5 and save the results with bounding boxes.
def detect_and_save_images(input_folder, output_folder, conf_threshold=0.3):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get list of image files in the input folder
    image_files = sorted(
        [Path(input_folder) / img for img in os.listdir(input_folder) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    )

    if not image_files:
        print("No images found in the input folder.")
        return

    for image_file in image_files:
        # Load image
        image = cv2.imread(str(image_file))
        if image is None:
            print(f"Error: Could not load {image_file}")
            continue

        print(f"Processing {image_file}...")

        # Run inference
        results = model(image)

        # Extract detections
        try:
            detections = results.xyxy[0].cpu().numpy()  # Bounding boxes
        except Exception as e:
            print(f"Error extracting detections from {image_file}: {e}")
            continue

        if detections.size == 0:
            print(f"No detections for {image_file}")
            continue

        # Draw bounding boxes on the image
        for *box, conf, cls in detections:
            if conf >= conf_threshold:
                x_min, y_min, x_max, y_max = map(int, box)
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                # Optionally, add the confidence and class labels
                label = f"{model.names[int(cls)]} {conf:.2f}"
                cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Save the processed image with bounding boxes
        output_image_path = Path(output_folder) / f"detected_{image_file.stem}.jpg"
        cv2.imwrite(str(output_image_path), image)
        print(f"Saved detected image as {output_image_path}")

# Folder containing the car plate images
input_folder = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates"
# Folder to save the detected images
output_folder = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images"

# Run the detection
detect_and_save_images(input_folder, output_folder, conf_threshold=0.3)

YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.12.4 torch-2.5.1 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 
/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Model loaded successfully.
Processing /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates/1.jpeg...
Saved detected image as /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images/detected_1.jpg
Processing /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates/10.jpeg...
Saved detected image as /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images/detected_10.jpg
Processing /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates/11.jpeg...


/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved detected image as /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images/detected_11.jpg
Processing /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates/12.jpeg...
Saved detected image as /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images/detected_12.jpg
Processing /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates/13.jpeg...


/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved detected image as /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images/detected_6.jpg
Processing /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates/7.jpeg...
Saved detected image as /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images/detected_7.jpg
Processing /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates/8.jpeg...


/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved detected image as /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images/detected_8.jpg
Processing /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates/9.jpeg...
Saved detected image as /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images/detected_9.jpg


/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


<b>Step 4.b: Display the Detected Objects</b>

In [7]:
import cv2
import matplotlib.pyplot as plt
import os
import math
import re

def natural_sort_key(s):
    # Extracts the numerical part from filenames for natural sorting
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def display_jpeg_images_in_grid(folder_path, images_per_row=4):
    # Collect only .jpeg and .jpg images and sort numerically based on extracted numbers
    image_paths = [os.path.join(folder_path, img) 
                   for img in sorted(os.listdir(folder_path), key=natural_sort_key) 
                   if img.endswith('.jpeg') or img.endswith('.jpg')]
    
    print(f"Found {len(image_paths)} images in the directory.")  # Debugging line

    if len(image_paths) == 0:
        print("No .jpeg or .jpg images found in the specified directory.")
        return

    # Load images without resizing
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is not None:
            images.append(img)
        else:
            print(f"Could not load image at {path}")

    # Calculate the number of rows needed based on images_per_row
    num_images = len(images)
    num_rows = math.ceil(num_images / images_per_row)

    # Display images in a grid format
    fig, axes = plt.subplots(num_rows, images_per_row, figsize=(20, 5 * num_rows))
    axes = axes.flatten()  # Flatten in case of multiple rows

    for i, img in enumerate(images):
        axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axes[i].axis('off')
        axes[i].set_title(f"Image {i + 1}")

    # Hide any extra subplots if there are fewer images than grid cells
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

display_jpeg_images_in_grid('/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Detected Images')

Found 43 images in the directory.


<b>Step 4.c: Crop the Car Images</b>

In [9]:
import torch
import cv2
import os

# Path to the YOLOv5 model directory and weights file
yolo_dir = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5"
model_path = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5m.pt"

# Crops the largest bounding box from the image based on area.
def crop_largest_box(image, detections):
    max_area = 0
    best_box = None

    # Iterate through detected boxes
    for detection in detections:
        xmin, ymin, xmax, ymax, confidence, cls = detection
        width = xmax - xmin
        height = ymax - ymin
        area = width * height

        # Update if this is the largest box found
        if area > max_area:
            max_area = area
            best_box = (int(xmin), int(ymin), int(xmax), int(ymax))

    # Crop the largest bounding box if found
    if best_box:
        xmin, ymin, xmax, ymax = best_box
        cropped_image = image[ymin:ymax, xmin:xmax]
        return cropped_image, best_box
    else:
        return None, None

# Processes images in the input folder, crops the largest detected bounding box, and saves the cropped images.
def process_images(input_folder, output_folder):
    
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get all image files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error: Could not load {image_path}")
            continue

        print(f"Processing {filename}...")

        # Run YOLOv5 for detection
        results = model(image)

        # Get bounding boxes (xyxy format: [xmin, ymin, xmax, ymax, confidence, class])
        detections = results.xyxy[0].cpu().numpy()

        # Crop the largest box
        cropped_image, best_box = crop_largest_box(image, detections)

        if cropped_image is not None:
            # Save the cropped image
            output_path = os.path.join(output_folder, f"cropped_{filename}")
            cv2.imwrite(output_path, cropped_image)
            print(f"Saved cropped image to {output_path}")
        else:
            print(f"No valid bounding boxes found for {filename}")

# Define input and output folders
input_folder = '/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Car Plates'
output_folder = '/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Cropped Plates'

# Process all images
process_images(input_folder, output_folder)


Processing 10.jpeg...


/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved cropped image to /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Cropped Plates/cropped_10.jpeg
Processing 26.jpeg...


/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved cropped image to /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Cropped Plates/cropped_26.jpeg
Processing 30.jpeg...


...

Saved cropped image to /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Cropped Plates/cropped_25.jpeg
Processing 33.jpeg...


/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved cropped image to /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Cropped Plates/cropped_33.jpeg
Processing 13.jpeg...
Saved cropped image to /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Cropped Plates/cropped_13.jpeg
Processing 29.jpeg...
Saved cropped image to /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Cropped Plates/cropped_29.jpeg
Processing 3.jpeg...


/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved cropped image to /Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/Cropped Plates/cropped_3.jpeg


<b>Step 5: Image Pre-processing<b>

In [10]:
import cv2
import os
from pathlib import Path

# Define the paths
input_folder = Path('/Users/raisa_hasan/Desktop/Computer Vision/Project/Cropped Plates')
output_folder = Path('/Users/raisa_hasan/Desktop/Computer Vision/Project/P Images')
output_folder.mkdir(exist_ok=True)  # Create output folder if it doesn't exist

# Prepares the image by applying Gaussian blur, CLAHE, and sharpening for OCR readability.
def preprocess_image_for_ocr(image):

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise slightly
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Enhance contrast using CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    contrast_enhanced = clahe.apply(blurred)

    return contrast_enhanced

# Preprocesses images in the input folder and saves the enhanced images to the output folder.
def preprocess_and_save_images(input_folder, output_folder):
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = input_folder / filename
            image = cv2.imread(str(image_path))
            
            if image is not None:
                processed_image = preprocess_image_for_ocr(image)
                
                # Save processed image to output folder
                output_path = output_folder / filename
                cv2.imwrite(str(output_path), processed_image)
                print(f"Saved preprocessed image to {output_path}")
            else:
                print(f"Error loading image: {image_path}")

# Run preprocessing and save images
preprocess_and_save_images(input_folder, output_folder)


Saved preprocessed image to /Users/aysham/Documents/Fall 2024/Computer Vision/Project/P Images/cropped_40.jpeg
Saved preprocessed image to /Users/aysham/Documents/Fall 2024/Computer Vision/Project/P Images/cropped_17.jpeg
Saved preprocessed image to /Users/aysham/Documents/Fall 2024/Computer Vision/Project/P Images/cropped_21.jpeg
Saved preprocessed image to /Users/aysham/Documents/Fall 2024/Computer Vision/Project/P Images/cropped_37.jpeg
Saved preprocessed image to /Users/aysham/Documents/Fall 2024/Computer Vision/Project/P Images/cropped_36.jpeg
Saved preprocessed image to /Users/aysham/Documents/Fall 2024/Computer Vision/Project/P Images/cropped_20.jpeg
Saved preprocessed image to /Users/aysham/Documents/Fall 2024/Computer Vision/Project/P Images/cropped_1.jpeg
Saved preprocessed image to /Users/aysham/Documents/Fall 2024/Computer Vision/Project/P Images/cropped_16.jpeg
Saved preprocessed image to /Users/aysham/Documents/Fall 2024/Computer Vision/Project/P Images/cropped_41.jpeg
Sa

<b>Step 6: EasyOCR to Detect Text on License Plates<b>

In [ ]:
import cv2
import os
import easyocr
import re
from pathlib import Path
import numpy as np

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)

# Paths
project_dir = Path("/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project")
raw_input_folder = project_dir / "Cropped Plates"  # Folder with raw images
preprocessed_folder = project_dir / "P Images"  # Folder with preprocessed images
output_folder = project_dir / "Processed Plates"  # Folder for successful OCR outputs
error_folder = project_dir / "Error Images"  # Folder for error images
log_file_path = project_dir / "EasyOcr.txt"  # File to log results

# Ensure output directories exist
preprocessed_folder.mkdir(exist_ok=True)
output_folder.mkdir(exist_ok=True)
error_folder.mkdir(exist_ok=True)

# Clear the EasyOcr.txt file if it exists
open(log_file_path, "w").close()

# Function to check if detected text is a valid plate format
def is_valid_plate(plate_text):
    return bool(re.fullmatch(r'\d{4,6}', plate_text))

# Function to apply sharpening to the image
def sharpen_image(image, strength):
    blurred = cv2.GaussianBlur(image, (3, 3), 0)
    return cv2.addWeighted(image, 1 + strength, blurred, -strength, 0)

# Function to run OCR on an image and return the first valid plate number
def run_ocr(image):
    ocr_results = reader.readtext(image, detail=0)
    valid_texts = [text for text in ocr_results if is_valid_plate(text)]
    return valid_texts[0] if valid_texts else None

# Function to apply EasyOCR and log results in order
def apply_easyocr_with_preprocessing(raw_folder, preprocessed_folder, output_folder, error_folder, log_file_path, resize_factor=0.5, sharpness_strength=1.5):
    # List and sort image files by numeric order
    raw_image_files = sorted(
        [f for f in os.listdir(raw_folder) if re.search(r'\d+', f)],
        key=lambda x: int(re.search(r'\d+', x).group())
    )

    # Dictionary to hold the final results for each image
    final_results = {}

    for filename in raw_image_files:
        raw_image_path = raw_folder / filename
        preprocessed_image_path = preprocessed_folder / filename
        image = cv2.imread(str(raw_image_path))

        if image is None:
            final_results[filename] = "NaN"
            error_path = error_folder / f"error_{filename}"
            cv2.imwrite(str(error_path), np.zeros((100, 100, 3), dtype=np.uint8))  # Save blank image if loading fails
            print(f"Error loading image: {raw_image_path}")
            continue

        # Attempt OCR on raw image
        detected_text = run_ocr(image)
        if detected_text:
            final_results[filename] = detected_text
            output_path = output_folder / f"ocr_{filename}"
            cv2.imwrite(str(output_path), image)
            print(f"Image: {filename} - Detected Plate Number (Raw): {detected_text}")
            continue

        # Apply preprocessing and retry OCR
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(gray_image, None, fx=resize_factor, fy=resize_factor)
        detected_text = run_ocr(resized_image)
        if detected_text:
            final_results[filename] = detected_text
            output_path = output_folder / f"ocr_{filename}"
            cv2.imwrite(str(output_path), resized_image)
            print(f"Image: {filename} - Detected Plate Number (Preprocessed): {detected_text}")
            continue

        # Apply sharpening and retry OCR
        sharpened_image = sharpen_image(resized_image, sharpness_strength)
        detected_text = run_ocr(sharpened_image)
        if detected_text:
            final_results[filename] = detected_text
            output_path = output_folder / f"ocr_{filename}"
            cv2.imwrite(str(output_path), sharpened_image)
            print(f"Image: {filename} - Detected Plate Number after sharpening: {detected_text}")
        else:
            final_results[filename] = "NaN"
            error_path = error_folder / f"error_{filename}"
            cv2.imwrite(str(error_path), image)
            print(f"Image: {filename} - No valid plate text detected after all attempts.")

    # Write the final results to the log file
    with open(log_file_path, "w") as log_file:
        # Sort by numeric order of filenames
        for filename, plate_number in sorted(final_results.items(), key=lambda x: int(re.search(r'\d+', x[0]).group())):
            log_file.write(f"Image: {filename} Licence plate Number: {plate_number}\n")

# Run the OCR process
apply_easyocr_with_preprocessing(raw_input_folder, preprocessed_folder, output_folder, error_folder, log_file_path)

Using CPU. Note: This module is much faster with a GPU.


Image: cropped_1.jpeg - Detected Plate Number (Raw): 934457
Image: cropped_2.jpeg - Detected Plate Number (Raw): 802694
Image: cropped_3.jpeg - No valid plate text detected after all attempts.
Image: cropped_4.jpeg - Detected Plate Number (Raw): 469534
Image: cropped_5.jpeg - Detected Plate Number (Preprocessed): 468442
Image: cropped_6.jpeg - Detected Plate Number (Preprocessed): 454950
Image: cropped_7.jpeg - Detected Plate Number (Raw): 922733
Image: cropped_8.jpeg - Detected Plate Number (Raw): 127928
Image: cropped_9.jpeg - Detected Plate Number (Raw): 597141
Image: cropped_10.jpeg - Detected Plate Number (Raw): 19884
Image: cropped_11.jpeg - Detected Plate Number (Raw): 790493
Image: cropped_12.jpeg - Detected Plate Number (Raw): 14475


<b>Step 7: Create carplates.txt<b>

In [ ]:
# Create the carplates.txt file
carplates_content = """
Image: 1 Licence plate Number: 934457
Image: 2 Licence plate Number: 802694
Image: 3 Licence plate Number: 485120
Image: 4 Licence plate Number: 469534
Image: 5 Licence plate Number: 468442
Image: 6 Licence plate Number: 454950
Image: 7 Licence plate Number: 922733
Image: 8 Licence plate Number: 127928
Image: 9 Licence plate Number: 597141
Image: 10 Licence plate Number: 19884
Image: 11 Licence plate Number: 790493
Image: 12 Licence plate Number: 14475
Image: 13 Licence plate Number: 498494
Image: 14 Licence plate Number: 294510
Image: 15 Licence plate Number: 75339
Image: 16 Licence plate Number: 546901
Image: 17 Licence plate Number: 279995
Image: 18 Licence plate Number: 626821
Image: 19 Licence plate Number: 843957
Image: 20 Licence plate Number: 419292
Image: 21 Licence plate Number: 534009
Image: 22 Licence plate Number: 534009
Image: 23 Licence plate Number: 574358
Image: 24 Licence plate Number: 37374
Image: 25 Licence plate Number: 341688
Image: 26 Licence plate Number: 754337
Image: 27 Licence plate Number: 644531
Image: 28 Licence plate Number: 667934
Image: 29 Licence plate Number: 823998
Image: 30 Licence plate Number: 108902
Image: 31 Licence plate Number: 532670
Image: 32 Licence plate Number: 751616
Image: 33 Licence plate Number: 585388
Image: 34 Licence plate Number: 224585
Image: 35 Licence plate Number: 809986
Image: 36 Licence plate Number: 674202
Image: 37 Licence plate Number: 362231
Image: 38 Licence plate Number: 50885
Image: 39 Licence plate Number: 401048
Image: 40 Licence plate Number: 67400
Image: 41 Licence plate Number: 498839
Image: 42 Licence plate Number: 644710
Image: 43 Licence plate Number: 4323
"""

file_path = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/carplates.txt"

# Write content to file
with open(file_path, "w") as file:
    file.write(carplates_content)

file_path

<b>Step 8: Supervised Learning<b>

In [33]:
import re
# Compare EasyOCR results with ground truth and classify results into TP, FP, TN, FN.
# Handles duplicate image entries by considering only the last processed entry for each image.
def compare_easyocr_with_ground_truth(ocr_results_path, ground_truth_path, calculation_file_path):
    try:
        # Color codes for colored messages
        GREEN = '\033[92m'
        RED = '\033[91m'
        RESET = '\033[0m'

        # Read files
        with open(ocr_results_path, 'r') as ocr_file:
            ocr_lines = ocr_file.readlines()
        with open(ground_truth_path, 'r') as gt_file:
            gt_lines = gt_file.readlines()

        # Process OCR results, keeping only the last valid entry for each image
        ocr_data = {}
        for line in ocr_lines:
            match = re.search(r'Image: (cropped_\d+\.(jpeg|jpg)).*?Licence plate Number: (\d+|NaN)', line)
            if match:
                image, _, plate = match.groups()
                ocr_data[image] = plate  # Overwrite to keep the last entry

        # Process ground truth
        gt_data = {}
        for line in gt_lines:
            match = re.search(r'Image: (\d+).*?Licence plate Number: (\d+)', line)
            if match:
                image, plate = match.groups()
                gt_data[f"cropped_{image}.jpeg"] = plate

        # Consolidate duplicate extensions in OCR data
        consolidated_ocr_data = {}
        for image, plate in ocr_data.items():
            base_name = re.sub(r'\.\w+$', '', image)  # Remove file extension
            consolidated_ocr_data[base_name] = plate

        # Create the calculation file
        with open(calculation_file_path, "w") as calc_file:
            for gt_image, gt_plate in gt_data.items():
                base_name = re.sub(r'\.\w+$', '', gt_image)
                ocr_plate = consolidated_ocr_data.get(base_name, "NaN")  # Default to NaN if not found

                if ocr_plate == gt_plate:
                    label = "TP"  # True Positive
                    message = f"Image {gt_image}: Success! EasyOCR result matches the ground truth: {gt_plate}"
                    print(GREEN + message + RESET)
                elif ocr_plate == "NaN":
                    label = "FN"  # False Negative
                    message = f"Image {gt_image}: Error! The plate number wasn't detected; the correct plate number is: {gt_plate}"
                    print(RED + message + RESET)
                elif len(ocr_plate) == len(gt_plate) and ocr_plate != gt_plate:
                    if all(ocr_plate[i] != gt_plate[i] for i in range(len(gt_plate))):
                        label = "FN"  # Completely incorrect plate
                        message = f"Image {gt_image}: Failed! Completely wrong number. Correct number is: {gt_plate}"
                        print(RED + message + RESET)
                    else:
                        label = "FP"  # Partial mismatch
                        mismatched_digits = [
                            idx + 1
                            for idx, (ocr_digit, gt_digit) in enumerate(zip(ocr_plate, gt_plate))
                            if ocr_digit != gt_digit
                        ]
                        message = (
                            f"Image {gt_image}: There is a mistake in digit(s) {', '.join(map(str, mismatched_digits))}. "
                            f"The correct plate number is: {gt_plate}"
                        )
                        print(RED + message + RESET)
                else:
                    label = "FP"  # Incorrect detection
                    message = f"Image {gt_image}: Failed! Incorrect plate number. Detected: {ocr_plate}, Correct: {gt_plate}"
                    print(RED + message + RESET)

                # Write the result to the calculation file
                calc_file.write(f"Image: {gt_image}, Car Plate: {gt_plate}, Detected: {ocr_plate}, Label: {label}\n")

    except Exception as e:
        print(RED + f"An error occurred: {e}" + RESET)


# Paths to the EasyOCR results, ground truth, and calculation file
ocr_results_path = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/EasyOcr.txt"
ground_truth_path = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/carplates.txt"
calculation_file_path = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/calculation.txt"

# Run the comparison
compare_easyocr_with_ground_truth(ocr_results_path, ground_truth_path, calculation_file_path)

Image cropped_1.jpeg: Success! EasyOCR result matches the ground truth: 934457
Image cropped_2.jpeg: Success! EasyOCR result matches the ground truth: 802694
Image cropped_3.jpeg: Success! EasyOCR result matches the ground truth: 485120
Image cropped_4.jpeg: Success! EasyOCR result matches the ground truth: 469534
Image cropped_5.jpeg: Success! EasyOCR result matches the ground truth: 468442
Image cropped_6.jpeg: Success! EasyOCR result matches the ground truth: 454950
Image cropped_7.jpeg: Success! EasyOCR result matches the ground truth: 922733
Image cropped_8.jpeg: Success! EasyOCR result matches the ground truth: 127928
Image cropped_9.jpeg: Success! EasyOCR result matches the ground truth: 597141
Image cropped_10.jpeg: Success! EasyOCR result matches the ground truth: 19884
Image cropped_11.jpeg: Success! EasyOCR result matches the ground truth: 790493
Image cropped_12.jpeg: Success! EasyOCR result matches the ground truth: 14475
Image cropped_13.jpeg: Success! EasyOCR result matc

<b>Step 9: Evaluation Matrix<b>

In [34]:
# Compute TP, TN, FP, FN, accuracy, precision, recall, and F1-score.
def evaluate_from_calculations(file_path):
    try:
        # Initialize counts for confusion matrix
        TP, TN, FP, FN = 0, 0, 0, 0

        # Read the calculation file
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Count occurrences of each label
        for line in lines:
            # True Positive: OCR detected the plate correctly, and it matches the ground truth
            if "Label: TP" in line:
                TP += 1
            # True Negative: No plate was present in the image, and OCR correctly identified this
            elif "Label: TN" in line:
                TN += 1
            # False Positive: OCR detected an incorrect plate (wrong digits or almost correct)
            elif "Label: FP" in line:
                FP += 1
            # False Negative: OCR failed to detect the plate number entirely
            elif "Label: FN" in line:
                FN += 1

        # Calculate evaluation metrics
        total = TP + TN + FP + FN
        accuracy = (TP + TN) / total if total > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Print the results
        print("Confusion Matrix:")
        print(f"True Positives (TP): {TP}")
        print(f"True Negatives (TN): {TN}")
        print(f"False Positives (FP): {FP}")
        print(f"False Negatives (FN): {FN}")
        print("\nEvaluation Metrics:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1_score:.4f}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Path to the calculation file
calculation_file_path = "/Users/raisa_hasan/Desktop/Web/Cpmputer_Vision_project/calculation.txt"
evaluate_from_calculations(calculation_file_path)

Confusion Matrix:
True Positives (TP): 38
True Negatives (TN): 0
False Positives (FP): 2
False Negatives (FN): 3

Evaluation Metrics:
Accuracy: 0.8837
Precision: 0.9500
Recall: 0.9268
F1-Score: 0.9383
